In [1]:
!pip install "unsloth[kaggle] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mcq9rll0/unsloth_dac7e356a3904b9cac80464d0e7838ac
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mcq9rll0/unsloth_dac7e356a3904b9cac80464d0e7838ac
  Resolved https://github.com/unslothai/unsloth.git to commit 9738cfdc526ccc50faa01260e8a64010b5c5be28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 14.0 MB/s eta 0:00

In [1]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-e9mcnz19
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-e9mcnz19
  Resolved https://github.com/huggingface/transformers.git to commit 2932f318a20d9e54cc7aea052e040164d85de7d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11588373 sha256=3542b8ea3846d3fb2d03880f8ac6e7d4a34114ea7ce9c85552f0c035d5f0c8b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ozf9afwg/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [2]:
!pip install trl

In [5]:
!pip install bitsandbytes
from unsloth import FastLanguageModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

In [7]:
from datasets import load_dataset

In [11]:
max_seq_length = 4096

In [12]:
dataset = load_dataset("imdb",split="train")

In [13]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype= None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.52.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "dense_h_to_4h", "dense_4h_to_h"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [20]:
trainer = SFTTrainer(
    model = model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=60,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="unsloth-test",
        optim="adamw_8bit",
        seed=3407
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/25000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shravyavenishetty03 (shravyavenishetty03-vaagdevi-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.113400
2,1.560400
3,2.009800
4,1.432000
5,1.472900
6,1.275800
7,1.950400
8,1.448400
9,2.791300
10,1.426400


TrainOutput(global_step=60, training_loss=0.8863704614962141, metrics={'train_runtime': 1303.0142, 'train_samples_per_second': 0.368, 'train_steps_per_second': 0.046, 'total_flos': 9227041964457984.0, 'train_loss': 0.8863704614962141})

In [21]:
inputs = tokenizer(
    [
        "I really like the movie because it shows emotions and talks humanity"
    ],
    return_tensors="pt",
).to("cuda")

In [22]:
inputs

{'input_ids': tensor([[128000,     40,   2216,   1093,    279,   5818,   1606,    433,   5039,
          21958,    323,  13739,  22706]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [30]:
# Generate output from model
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode and clean the output
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
cleaned = [text.replace("\n", " ").strip() for text in decoded]

# Print the cleaned result
print(cleaned[0])

I really like the movie because it shows emotions and talks humanity. I think it is a very good movie, especially for those who love to see movies that show human emotion. It's a pretty good movie. It's a great story about people. It has a lot of beautiful moments in it.


In [31]:
outputs

tensor([[128000,     40,   2216,   1093,    279,   5818,   1606,    433,   5039,
          21958,    323,  13739,  22706,     13,    358,   1781,    433,    374,
            264,   1633,   1695,   5818,     11,   5423,    369,   1884,    889,
           3021,    311,   1518,   9698,    430,   1501,   3823,  20356,    627,
           2181,    596,    264,   5128,   1695,   5818,     13,   1102,    596,
            264,   2294,   3446,    922,   1274,     13,   1102,    706,    264,
           2763,    315,   6366,  14269,    304,    433,     13, 128001]],
       device='cuda:0')

In [32]:
tokenizer.batch_decode(outputs)

["<|begin_of_text|>I really like the movie because it shows emotions and talks humanity. I think it is a very good movie, especially for those who love to see movies that show human emotion.\nIt's a pretty good movie. It's a great story about people. It has a lot of beautiful moments in it.<|end_of_text|>"]

In [33]:
model.save_pretrained("lora_model")

In [35]:
!ls -R /content

/content:
huggingface_tokenizers_cache  sample_data	      unsloth-test
lora_model		      unsloth_compiled_cache  wandb

/content/huggingface_tokenizers_cache:
models--unsloth--llama-3-8b-bnb-4bit

/content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit:
blobs  refs  snapshots

/content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs:
6b7e24ffc27d3786a23045df96fc0bbbe02ce9d3
9c44d1d78fb1f530d4839937d9850f6f5621206d
b197f72effb9d5ed16ee0f5663e11e4cfac2ba62

/content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/refs:
main

/content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/snapshots:
f20166737ba79ae1129e83194c84e2ec0013bf1b

/content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/snapshots/f20166737ba79ae1129e83194c84e2ec0013bf1b:
special_tokens_map.json  tokenizer_config.json	tokenizer.json

/content/lora_model:
adapter_config.json  adapter_model.safetensors	README.md

/content/samp

In [36]:
# Zip all required folders
!zip -r /content/all_colab_folders.zip \
  /content/huggingface_tokenizers_cache \
  /content/lora_model \
  /content/sample_data \
  /content/unsloth-test \
  /content/unsloth_compiled_cache \
  /content/wandb

# Download the zip file
from google.colab import files
files.download('/content/all_colab_folders.zip')

  adding: content/huggingface_tokenizers_cache/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/6b7e24ffc27d3786a23045df96fc0bbbe02ce9d3 (deflated 96%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/b197f72effb9d5ed16ee0f5663e11e4cfac2ba62 (deflated 74%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/9c44d1d78fb1f530d4839937d9850f6f5621206d (deflated 61%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/snapshots/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/snapshots/f20166737ba79ae1129e83194c84e2ec0013bf1b/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>